In [1]:
%load_ext Cython

In [2]:
%%cython --annotate

cimport cython
from libc.math cimport pi, sqrt, exp, pow, fabs


@cython.cdivision(True)
def normpdf(double x):
    return 1 / sqrt(2 * pi) * exp(-pow(x, 2) / 2)

@cython.cdivision(True)
def density_upper_cython(double t, double mu, double a, double b, double x0, unsigned int trunc_num=100):
    """
    First passage time density on the upper boundary
    """
    cdef double tau, a1, factor, result, rj, term
    cdef unsigned int j
    tau = mu + b
    a1 = a - x0
    factor = pow(t, -1.5) * exp(a1 * tau - 0.5 * pow(tau, 2) * t - b / (2 * a) * pow(a1, 2)) / sqrt(2 * pi)
    result = 0
    for j in range(trunc_num):
        rj = (2 * j + 1) * a - pow(-1, j) * x0
        term = pow(-1, j) * rj * exp(0.5 * (b / a - 1 / t) * pow(rj, 2))
        if fabs(term) < 1e-20:
            break
        result += term
    return result * factor

@cython.cdivision(True)
def density_lower_cython(double t, double mu, double a, double b, double x0, unsigned int trunc_num=100):
    """
    First passage time density on the upper boundary
    """
    cdef double tau, a1, factor, result, rj, term
    cdef unsigned int j
    tau = -mu + b
    a1 = a + x0
    factor = pow(t, -1.5) * exp(a1 * tau - 0.5 * pow(tau, 2) * t - b / (2 * a) * pow(a1, 2)) / sqrt(2 * pi)
    result = 0
    for j in range(trunc_num):
        rj = (2 * j + 1) * a + pow(-1, j) * x0
        term = pow(-1, j) * rj * exp(0.5 * (b / a - 1 / t) * pow(rj, 2))
        if fabs(term) < 1e-20:
            break
        result += term
    return result * factor


@cython.cdivision(True)
def density_vertical_cython(double x, double mu, double a, double b, double x0, double T, unsigned int trunc_num=100):
    """
    exit density on the vertical boundary
    """
    cdef double t1, t2, t3, t4, term, factor
    cdef unsigned int j
    x = x - x0
    factor = exp(mu * x - 0.5 * pow(mu, 2) * T) / sqrt(T)
    result = 1 / sqrt(2 * pi) * exp(-pow(x, 2) / (2*T))
    for j in range(1, trunc_num):
        t1 = 4 * b * j * (2 * a * j + x0) - pow(x - 4 * a * j, 2) / (2 * T)
        t2 = 4 * b * j * (2 * a * j - x0) - pow(x + 4 * a * j, 2) / (2 * T)
        t3 = 2 * b * (2 * j - 1) * (2 * a * j - a + x0) - pow(x + (4 * j - 2) * a + 2 * x0, 2) / (2 * T)
        t4 = 2 * b * (2 * j - 1) * (2 * a * j - a - x0) - pow(x - (4 * j - 2) * a + 2 * x0, 2) / (2 * T)
        term = exp(t1) + exp(t2) - exp(t3) - exp(t4)
        if fabs(term) < 1e-20:
            break
        result += term / sqrt(2 * pi)
    return result * factor

In [3]:
mu = 1.0
a = 1.5
b = 0.3
x0 = -0.1

t = 1.0
x = 0

%timeit density_upper_cython(t, mu, a, b, x0)
%timeit density_lower_cython(t, mu, a, b, x0)
%timeit density_vertical_cython(x, mu, a, b, x0, t)


463 ns ± 6.29 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
464 ns ± 4.58 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
400 ns ± 5.12 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
density_upper_cython(t, mu, a, b, x0), density_lower_cython(t, mu, a, b, x0), density_vertical_cython(x, mu, a, b, x0, t)

(0.6081958059989745, 0.0614835657319015, 0.25112391101626685)

In [5]:
import time
from hall1997_py import *

start1 = time.time()
for i in range(1000):
    result1 = density_upper_cython(t, mu, a, b, x0)
end1 = time.time()
elapsed_time1 = (end1 - start1) * 10**9 / 1000

start2 = time.time()
for i in range(1000):
    result2 = density_lower_cython(t, mu, a, b, x0)
end2 = time.time()
elapsed_time2 = (end2 - start2) * 10**9 / 1000

start3 = time.time()
for i in range(1000):
    result3 = density_vertical_cython(x, mu, a, b, x0, t)
end3 = time.time()
elapsed_time3 = (end3 - start3) * 10**9 / 1000

print("Result:", result1, result2, result3)
print("Execution Time:", elapsed_time1, elapsed_time2, elapsed_time3, "nanoseconds")

Result: 0.6081958059989745 0.0614835657319015 0.25112391101626685
Execution Time: 556.7073822021484 535.4881286621094 471.5919494628906 nanoseconds


In [6]:
import time
from hall1997_py import *

start1 = time.time()
for i in range(1000):
    result1 = density_upper(t, mu, a, b, x0)
end1 = time.time()
elapsed_time1 = (end1 - start1) * 10**9 / 1000

start2 = time.time()
for i in range(1000):
    result2 = density_lower(t, mu, a, b, x0)
end2 = time.time()
elapsed_time2 = (end2 - start2) * 10**9 / 1000

start3 = time.time()
for i in range(1000):
    result3 = density_vertical(x, mu, a, b, x0, t)
end3 = time.time()
elapsed_time3 = (end3 - start3) * 10**9 / 1000

print("Result:", result1, result2, result3)
print("Execution Time:", elapsed_time1, elapsed_time2, elapsed_time3, "nanoseconds")

Result: 0.6081958059989745 0.0614835657319015 0.25112391101626685
Execution Time: 4042.1485900878906 4040.241241455078 6007.194519042969 nanoseconds
